In [1]:
%load_ext autoreload
%autoreload 2

### Promblema 
El problema onsiste en predecir la probabilidad de atraso de los vuelos que aterrizan o despegan del aeropuerto de Santiago de Chile (SCL). Para eso les entregamos un dataset usando datos públicos y reales donde cada fila corresponde a un vuelo que aterrizó o despegó de SCL.

1. ¿Cómo se distribuyen los datos? ¿Qué te llama la atención o cuál es tu conclusión sobre esto?

In [2]:
# importar librerias
##generacion de path
import latam_airlines.utils.paths as path
## funciones propias
from latam_airlines.utils.latam_utils import check_quality
## manipulacion de datos
import pandas as pd
import numpy as np
## Visualización de los datos
import matplotlib.pyplot as plt
import seaborn as sns

### Lectura y calidad de datos

In [3]:
# Lectura del archivo dataset_SCL.csv
dataset_SCL = pd.read_csv(path.data_raw_dir('dataset_SCL.csv'), low_memory=False)
dataset_SCL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Fecha-I    68206 non-null  object 
 1   Vlo-I      68167 non-null  object 
 2   Ori-I      68167 non-null  object 
 3   Des-I      68167 non-null  object 
 4   Emp-I      68167 non-null  object 
 5   Fecha-O    68167 non-null  object 
 6   Vlo-O      68166 non-null  object 
 7   Ori-O      68167 non-null  object 
 8   Des-O      68167 non-null  object 
 9   Emp-O      68167 non-null  object 
 10  DIA        68167 non-null  float64
 11  MES        68167 non-null  float64
 12  AÑO        68167 non-null  float64
 13  DIANOM     68167 non-null  object 
 14  TIPOVUELO  68167 non-null  object 
 15  OPERA      68167 non-null  object 
 16  SIGLAORI   68167 non-null  object 
 17  SIGLADES   68167 non-null  object 
dtypes: float64(3), object(15)
memory usage: 9.4+ MB


observamos que:
- En total hay 68206 datos(ojo la fila 0 cuenta como dato)
- Hay columnas que no tiene todas las filas, indicando la existencia de valores faltantes
- Es necesario transformar las columnas de fecha a formato fecha
- Vemos que no consume mucha uso de memoria, por esta razon crearemos una copia para prevenir futuros problemas al cometer algun error.
- No existe nuestr variable target u objetivo, lo que se traduce en featuring data para crear el target

In [4]:
dataset_SCL_copy = dataset_SCL.copy()
dataset_SCL.sample(10)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
57319,2017-11-12 17:00:00,106,SCEL,SCSE,SKU,2017-11-12 17:00:00,106,SCEL,SCSE,SKU,12.0,11.0,2017.0,Domingo,N,Sky Airline,Santiago,La Serena
38249,2017-07-31 11:00:00,200,SCEL,SPJC,LAW,2017-07-31 12:15:00,200,SCEL,SPJC,JMR,31.0,7.0,2017.0,Lunes,I,Latin American Wings,Santiago,Lima
34681,2017-07-21 20:10:00,503,SCEL,SAEZ,SKU,2017-07-21 20:43:00,503,SCEL,SAEZ,SKU,21.0,7.0,2017.0,Viernes,I,Sky Airline,Santiago,Buenos Aires
10640,2017-02-15 09:55:00,504,SCEL,KMIA,LAN,2017-02-15 10:00:00,504,SCEL,KMIA,LAN,15.0,2.0,2017.0,Miercoles,I,Grupo LATAM,Santiago,Miami
14786,2017-03-11 09:55:00,231,SCEL,SCQP,LAN,2017-03-11 09:56:00,231,SCEL,SCQP,LXP,11.0,3.0,2017.0,Sabado,N,Grupo LATAM,Santiago,Temuco
121,2017-01-14 21:10:00,93,SCEL,CYYZ,ACA,2017-01-14 22:49:00,93,SCEL,CYYZ,ACA,14.0,1.0,2017.0,Sabado,I,Air Canada,Santiago,Toronto
36813,2017-07-20 01:50:00,380,SCEL,SCAR,LAN,2017-07-20 01:48:00,380,SCEL,SCAR,LXP,20.0,7.0,2017.0,Jueves,N,Grupo LATAM,Santiago,Arica
59598,2017-11-26 09:31:00,300,SCEL,SCSE,LAN,2017-11-26 10:23:00,300,SCEL,SCSE,LXP,26.0,11.0,2017.0,Domingo,N,Grupo LATAM,Santiago,La Serena
46737,2017-09-20 06:00:00,142,SCEL,SCCF,LAN,2017-09-20 06:02:00,142,SCEL,SCCF,LAN,20.0,9.0,2017.0,Miercoles,N,Grupo LATAM,Santiago,Calama
52464,2017-10-30 09:30:00,146,SCEL,SCCF,LAN,2017-10-30 09:36:00,146,SCEL,SCCF,LAN,30.0,10.0,2017.0,Lunes,N,Grupo LATAM,Santiago,Calama


### Verificacion de calidad de datos

In [5]:
check_quality(dataset_SCL)

Valores NA por columna:
Fecha-I       0
Vlo-I        39
Ori-I        39
Des-I        39
Emp-I        39
Fecha-O      39
Vlo-O        40
Ori-O        39
Des-O        39
Emp-O        39
DIA          39
MES          39
AÑO          39
DIANOM       39
TIPOVUELO    39
OPERA        39
SIGLAORI     39
SIGLADES     39
dtype: int64

Cantidad de datos duplicados:
0

Valores únicos por columna:
Fecha-I      53271
Vlo-I          581
Ori-I            1
Des-I           61
Emp-I           30
Fecha-O      62742
Vlo-O          857
Ori-O            1
Des-O           60
Emp-O           32
DIA             31
MES             12
AÑO              2
DIANOM           7
TIPOVUELO        2
OPERA           23
SIGLAORI         1
SIGLADES        59
dtype: int64


Verificando la calidad de los datos podemos observar:
- existen 39 filas con dainformacion NAN que se pueden iliminar.
- No hay datos duplicados.
- es necesario hacer el cambio de variables string a catgoricas.
- cambiar la columnas fecha a objeto fecha.

In [6]:
# Eliminamos valores NA
dataset_SCL.dropna(inplace=True)

In [7]:
dataset_SCL.sample(10)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
40204,2017-08-20 17:00:00,134,SCEL,SCCF,SKU,2017-08-20 16:58:00,134,SCEL,SCCF,SKU,20.0,8.0,2017.0,Domingo,N,Sky Airline,Santiago,Calama
37512,2017-07-05 22:05:00,602,SCEL,KLAX,LAN,2017-07-05 22:05:00,602,SCEL,KLAX,LAN,5.0,7.0,2017.0,Miercoles,I,Grupo LATAM,Santiago,Los Angeles
30211,2017-06-13 17:30:00,215,SCEL,SCIE,LAN,2017-06-13 17:26:00,215,SCEL,SCIE,LXP,13.0,6.0,2017.0,Martes,N,Grupo LATAM,Santiago,Concepcion
35777,2017-07-19 19:45:00,219,SCEL,SCIE,LAN,2017-07-19 19:41:00,219,SCEL,SCIE,LXP,19.0,7.0,2017.0,Miercoles,N,Grupo LATAM,Santiago,Concepcion
50888,2017-10-30 15:45:00,5,SCEL,SCCI,SKU,2017-10-30 15:43:00,5,SCEL,SCCI,SKU,30.0,10.0,2017.0,Lunes,N,Sky Airline,Santiago,Punta Arenas
29542,2017-06-17 09:00:00,114,SCEL,SCAT,LAN,2017-06-17 09:01:00,114,SCEL,SCAT,LAN,17.0,6.0,2017.0,Sabado,N,Grupo LATAM,Santiago,Copiapo
60830,2017-11-22 15:00:00,642,SCEL,SPJC,LAN,2017-11-22 15:10:00,642,SCEL,SPJC,LAN,22.0,11.0,2017.0,Miercoles,I,Grupo LATAM,Santiago,Lima
8246,2017-02-07 18:55:00,531,SCEL,SACO,SKU,2017-02-07 18:56:00,531,SCEL,SACO,SKU,7.0,2.0,2017.0,Martes,I,Sky Airline,Santiago,Cordoba
32306,2017-06-16 11:00:00,200,SCEL,SPJC,LAW,2017-06-16 11:25:00,200,SCEL,SPJC,JMR,16.0,6.0,2017.0,Viernes,I,Latin American Wings,Santiago,Lima
8343,2017-02-21 05:55:00,802,SCEL,SPJC,SKU,2017-02-21 07:26:00,802,SCEL,SPJC,SKU,21.0,2.0,2017.0,Martes,I,Sky Airline,Santiago,Lima


In [10]:
cols_date = ['Fecha-I','Fecha-O']
for col in cols_date:
    dataset_SCL[col] = pd.to_datetime(dataset_SCL[col], format='%Y-%m-%d %H:%M:%S')

In [11]:
dataset_SCL.describe(datetime_is_numeric=True)

,Fecha-I,Fecha-O,DIA,MES,AÑO
count,68166,68166,68166.000000,68166.000000,68166.000000
mean,2017-07-05 10:37:37.216794368,2017-07-05 10:46:43.755831296,15.714858,6.620441,2017.000029
min,2017-01-01 00:15:00,2017-01-01 00:04:00,1.000000,1.000000,2017.000000
25%,2017-03-31 12:05:00,2017-03-31 12:16:00,8.000000,3.000000,2017.000000
50%,2017-07-10 01:55:00,2017-07-10 01:48:00,16.000000,7.000000,2017.000000
75%,2017-10-08 08:26:00,2017-10-08 08:23:00,23.000000,10.000000,2017.000000
max,2017-12-31 23:55:00,2018-01-01 00:12:00,31.000000,12.000000,2018.000000
std,NaN,NaN,8.782890,3.522743,0.005417


observamos que:
- La resolucion temporal es de un anio de datos, puntualmente el anio 2017.
- No podemos obtener conclusiones o analizar datos con los dias, meses o anios en formato numero, por que son variables categoricas ordinales,esto al poseer un orden natural.
- Debemos crear o reemplazar las columnas de dia, mes y anio(no tan relevante) con sus respectivo datos categoricos.
- Vemos que las demas variables son string y veremos si es necesario cambiarlas a categorias.
- hasta ahora no se eliminaran columns debido que seran utilies mas adelante para hacer procesamientos.

In [12]:
# creamos variables para ayudarnos a hacer el analisis
days = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
months = ['Ene','Feb','Mar','Abr','May','Jun','Jul','Ago','Sep','Oct','Nov','Dic']
d = dict(zip(np.arange(1, 13),months))
dataset_SCL['MESNOM'] = dataset_SCL['MES'].astype(int).replace(d, regex=True)

#cambiamos las demas variables a formato
cols = ['Vlo-I', 'Ori-I', 'Des-I', 'Emp-I','Vlo-O','Ori-O', 'Des-O', 'Emp-O','DIANOM', 'TIPOVUELO','OPERA', 'SIGLAORI', 'SIGLADES']
for col in cols:
    if col == 'DIANOM':
        dataset_SCL[col] = pd.Categorical(dataset_SCL[col],ordered=True, categories=days)
    else:
        dataset_SCL[col] = dataset_SCL[col].astype('category')

In [13]:
dataset_SCL.sample(5)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,MESNOM
54742,2017-10-15 14:55:00,642,SCEL,SPJC,LAN,2017-10-15 15:19:00,642,SCEL,SPJC,LAN,15.0,10.0,2017.0,Domingo,I,Grupo LATAM,Santiago,Lima,Oct
10869,2017-02-21 07:40:00,634,SCEL,SPJC,LAN,2017-02-21 07:38:00,634,SCEL,SPJC,LAN,21.0,2.0,2017.0,Martes,I,Grupo LATAM,Santiago,Lima,Feb
61390,2017-11-19 20:05:00,7813,SCEL,SABE,LAN,2017-11-19 20:20:00,7813,SCEL,SABE,DSM,19.0,11.0,2017.0,Domingo,I,Grupo LATAM,Santiago,Buenos Aires,Nov
20636,2017-04-09 20:10:00,324,SCEL,SCFA,LAN,2017-04-09 20:17:00,324,SCEL,SCFA,LXP,9.0,4.0,2017.0,Domingo,N,Grupo LATAM,Santiago,Antofagasta,Abr
42942,2017-08-23 09:45:00,455,SCEL,SABE,LAN,2017-08-23 09:51:00,455,SCEL,SABE,LAN,23.0,8.0,2017.0,Miercoles,I,Grupo LATAM,Santiago,Buenos Aires,Ago


Con la transformacion de las variables al formato correspondiente iniciamos el analisis de la informacion. Sin embargo, como el objetivo es predecir el retraso en los vuelos de llegada o salida del aeropuerto es fundamenta crear nuestra variable objetivo o target y otras variables auxiliares, ademas, con el fin de no divagar en los datos o obtener metricas que no esten orientadas a responder nuestro objetivo principal(probabilidad de retraso), aqui abordaremos el segundo punto del desafio.